In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import pymn
import anndata as ad
import time
import datetime
import os
from pyprojroot import here
import resource

In [ ]:
here()
start_time = time.time()

base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [ ]:
merged = sc.read_h5ad(base_data_folder + '/whole_mouse_brain/processed/merged_Zeng_AWS.Oct2023.h5ad', backed=True)

In [ ]:
merged.obs['cell.type'] = merged.obs['cell.type'].astype(str)
merged.obs['study_id'] = merged.obs['study_id'].astype(str)

In [ ]:
#filter for cells of Zeng
merged = merged[merged.obs['study_id']=="Zeng"]

In [ ]:
%%time
merged_in_memory = merged.to_memory() #9 minutes on an inkwell, ~60Gb RAM

In [ ]:
%%time
ptrained_Zeng = pymn.trainModel(merged_in_memory, 'study_id', 'cell.type')

In [ ]:
ptrained_Zeng.to_csv(base_data_folder + '/whole_mouse_brain/processed/zeng/merged_Zeng_AWS.Oct2023.pretrained_Zeng.csv.gz', compression = 'gzip')

In [ ]:
#Minor code duplication for Macosko, may need a restart to free up memory

In [ ]:
merged = sc.read_h5ad(base_data_folder + '/whole_mouse_brain/processed/merged_Zeng_AWS.Oct2023.h5ad', backed=True)
merged.obs['cell.type'] = merged.obs['cell.type'].astype(str)
merged.obs['study_id'] = merged.obs['study_id'].astype(str)

In [ ]:
#filter for cells
merged = merged[merged.obs['study_id']=="Macosko"]

In [ ]:
%%time
merged_in_memory = merged.to_memory()

In [ ]:
%%time
ptrained_Mac = pymn.trainModel(merged_in_memory, 'study_id', 'cell.type')

In [ ]:
ptrained_Mac.to_csv(base_data_folder + '/whole_mouse_brain/processed/macosko/merged_Zeng_AWS.Oct2023.pretrained_Macosko.csv.gz', compression = 'gzip')

In [ ]:
###########################################
#the 109 BARseq genes
###########################################

In [ ]:
barseq_genes = pd.read_csv(base_data_folder + "/whole_mouse_brain/processed/barseq/barseq_gene_ids.csv")

In [ ]:
#needs the original data as only 90 of 109 overlap with HVGs
len(set(barseq_genes["ENSEMBL"].tolist()).intersection(merged.var.index.tolist()))

In [ ]:
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/macosko/subsets/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.barseq_109.h5ad")
adata_zeng = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.barseq_109.h5ad")

In [ ]:
adata_zeng.obs["cell.type"] = adata_zeng.obs.cl
adata_zeng.obs["study_id"] = "Zeng"

In [ ]:
adata_macosko.var['highly_variable'] = True
adata_zeng.var['highly_variable'] = True

In [ ]:
%%time
ptrained_Mac_109 = pymn.trainModel(adata_macosko, 'study_id', 'cell.type')

In [ ]:
ptrained_Mac_109.to_csv(base_data_folder + '/whole_mouse_brain/processed/macosko/Macosko_Mouse_Atlas_Single_Nuclei.pretrained_Macosko_109_genes.csv.gz', compression = 'gzip')

In [ ]:
%%time
ptrained_Zeng_109 = pymn.trainModel(adata_zeng, 'study_id', 'cell.type')

In [ ]:
ptrained_Zeng_109.to_csv(base_data_folder + '/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome.pretrained_Zeng_109_genes.csv.gz', compression = 'gzip')